# Gaussian KD-Tree

### Quelle: http://graphics.stanford.edu/papers/gkdtrees/gkdtrees.pdf

## Variabeln:

$n$ Anzahl der zu verarbeiten Werte

$d$ Dimension des VR (=5)

$v_i$ Wert eines Pixels an einer Stelle

$p_i$ Position eines Pixels (siehe 2.5)

$c_i$ Farbe eines Pixels

$\delta$ Standardabweichung (Standardmäßig = 1)

$s$ Anzahl der Sample (Stichproben)

$\delta_{s}$ größe des Gaussian Kernels

## Mathe:


***(1) Allgemeine Formel (linear Kombination des Wertes):***

$$\hat v_i = \sum^{n}_{j=1} \lambda_{ij} v_i$$

***(2) bilateraler Filter ausgedrückt:***
$$\hat v_i = \sum^{n}_{j=1} e^{\frac{-|p_i-p_j|^2}{2 \delta^2_p}} e^{\frac{-|c_i - c_j|^2}{2 \delta_c^2}} v_i$$
***(2.5) vereinfachen durch das einschließen der Farbe in $p_i$ als 5d VR (x, y, r, g, b) und  $\delta_p$ Standardwert als 1 wählen:***
$$\hat v_i = \sum^{n}_{j=1} e^{\frac{-|p_i-p_j|^2}{2}} v_i$$

***Beschleunigungen der Berechnung:***
* $|p_i-p_j| > 3 * standardabweichung$ $\delta$ wird ignoriert
* KD-Tree speicher Struktur
* wählen von weniger Werten als $n$ für den KD_Tree
* down- und absampleing

## KD-Tree

### Node-Inhalt 

* $\eta_{d}$ Dimension die der Node beschneidet
* $\eta_{cut}$ Wert an dem die Dimension beschnitten wird
* $\eta_{left}$, $\eta_{right}$ beiden folgende Nodes
* $\eta_{min}$, $\eta_{max}$ beschränkung der Bounding-Box (berechnet durch die vorherigen Nodes)

  $\eta_{max} =$ min(aller $\eta_{cut}$ der Vorfahren mit selber Dimension, aber größerem Wert)
  
  $\eta_{min} =$ max(aller $\eta_{cut}$ der Vorfahren mit selber Dimension, aber kleinerem Wert)

### LeafNode-tinhalt
* $p_i$ Position in d Dimension

### Konstruktion

Ausgehend vom 5d VR (x, y, r, g, b) wird von einem Punkt gestartet und dann nahe liegennde Werte gewählt, damit jedes Blat max. $\delta_{s}/2$ entfernt ist.
1. Zu $p_i$ wird die Bounding Box aller Werte berechnet, wenn die Diagonal < als $\delta_{s}$ ist:
    * dann wird ein LeafNode erstellt und der Mittelpunkt der Bounding Box gespeichert (Median geht auch, ist aber laut Quelle gleichschnell)
    * sonst wird die Längste Seite und die Eingabelist halbiert und rekursiv fortgesetz (neue Nodes ab 1)

### Finden eines Werts

#### Input:
* $q$ Position (x, y, r, g, b)
* $\delta$ Standard-Abweichung
* $s$ Anzahl an Samples

#### Output:
* list an $s$ Position $p_{i}$ mit der entsprechenden Gewichtung $w_{i}$

***dabei ist jeder Wert gleich Wahrscheinlich getroffen zu werden, wenn mehr als $s$ möglich sind***

#### Berechnung:
CDF-Funktion = Normalverteilung

$$cdf(x) = \frac{1}{\sqrt{2\pi}} \int_\infty^x \mathrm{e}^{-\frac{x^2}{2}}\,\mathrm{d}x$$

1. 
erwartete Anzahl an Linken Position:
$$left_{temp} = \frac{cdf(\eta_{cut} - q) - cdf(\eta_{min} - q)}{cdf(\eta_{max} - q) - cdf(\eta_{min} - q)} * s $$
$$left = \lfloor left_{temp} \rfloor$$

erwartete Anzahl an rechten Position:
$$right = \lfloor s - left_{temp} \rfloor$$

2.
Falls $sample$ $>$ $left$ $+$ $right$, dann wird zufällig entweder $left$ der $right$ um 1 erhöht (Rundungsfehler ausgleichen)

3.
Wenn eins $\ne 0$ dann dort bei dem Node wieder holen, bis ein LeafNode erreicht ist

4.
Beim LeafNode:
Position zurückgebn und

$$w_{i} = e^{\frac{-|q-p_i|^2}{2\delta}} (< 3 * \delta)$$

## GPU Implementation

Der KD_Tree muss auf der CPU gebaut werden
Danach ist die Query Methode auf die GPU implementierbar, weil die Daten gleichbleiben

### Probleme:
1. Die Implementation des KD-Tree auf der GPU ist schwierig, weil es keine Rekursion gibt
2. Die Query Methode muss iterativ implementiert werden, weil die Cuda keinen Call Stack hat.
3. Die GPU kann nicht genug Speicherplatz haben

### Lösungen:
1. Die CPU übernimmt Anfangs die groben Schritte rekursiv mit GPU unterstützung beim bauen der Bounding Boxen. Wenn genügend Berechnungen gebraucht werden, dann wird der Ebenen nach der Baum gebeaut. Dabei übernimmt jeder Kernel (thread blocks) einen Build-Job. Dieser besteht aus einem Pointer zum Vorfahren und einem Array an Positionen $p_i$. Am Ende wird der Baum von unten nach oben durchlaufen um $n_{max}$ und $n_{min}$ zu berechnen

2. Die Query methode lässt sich iterativ auf der GPU implementieren, in dem man übergabe Argument in gemeinsamen Speicher von CPU und GPU speichert. Wenn ein normaler Node getroffen wird, dann geht der Thread den Baum runter und gibt den die andere hälfte (wenn gebraucht) in eine gemeinsame ***pending Work-Structur***. Wenn ein LeafNode ereicht wird, dann wird mit atomaren floating Points auf ihn zugegriffen

3. Man kann den KD-Tree aufteilen und jedes Stück einzeln bearbeiten oder Stochastisch den Baum abspeichern, das Problem kann dann aber trotzdem auftreten